# Analyse des Résultats de Génération de Texte

Ce notebook a pour objectif de charger, d'analyser et de visualiser les résultats des différentes expériences de génération de texte. 

Nous allons :
1.  Charger tous les fichiers de résultats `.json`.
2.  Compiler les métriques (Cohérence, Diversité, MAUVE, Longueur) dans un tableau unique.
3.  Générer des graphiques comparatifs pour visualiser les forces et faiblesses de chaque stratégie de décodage.

## 1. Installation et Imports

Assurez-vous que les bibliothèques nécessaires sont installées. `pandas` est utilisé pour la manipulation des données et `matplotlib` pour les graphiques.

In [1]:
!pip install pandas matplotlib

In [46]:
import json
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

## 2. Chargement et Traitement des Données

Les fonctions ci-dessous parcourent les sous-dossiers, trouvent tous les fichiers de résultats, extraient les informations pertinentes (dataset, stratégie) du nom du fichier, et chargent les métriques depuis le contenu JSON. Le tout est ensuite assemblé dans un DataFrame `pandas`.

In [47]:
def parse_filename(filepath):
    """Analyse le nom du fichier pour extraire le dataset et la stratégie."""
    basename = os.path.basename(filepath)
    
    # Isole la partie principale du nom de fichier, ex: 'book_greedy_gpt2-xl_256'
    if '_gpt2-xl_256' in basename:
        core_name = basename.split('_gpt2-xl_256')[0]
    else:
        core_name = basename.rsplit('_result.json', 1)[0] # Fallback

    parts = core_name.split('_')
    dataset = parts[0]
    strategy = '_'.join(parts[1:])

    # Nettoyage des noms de stratégie pour l'affichage
    strategy_map = {
        'p-0.95': 'Nucleus (p=0.95)',
        'typical-0.95': 'Typical (p=0.95)',
        'greedy': 'Greedy Search'
    }
    return dataset, strategy_map.get(strategy, strategy)

def load_all_results(base_dir='open_text_gen'):
    """Charge tous les résultats d'évaluation et les fusionne."""
    all_results_map = {}

    # Trouve tous les fichiers de résultats de diversité/mauve/longueur
    result_files = glob.glob(os.path.join(base_dir, '**/*_diversity_mauve_gen_length_result.json'), recursive=True)

    for f_path in result_files:
        try:
            dataset, strategy = parse_filename(f_path)
            with open(f_path, 'r') as f:
                data = json.load(f)[0]
            
            key = (dataset, strategy)
            all_results_map[key] = {
                'Dataset': dataset,
                'Strategy': strategy,
                'MAUVE': data['mauve_score'],
                'Gen_Length': data['prediction_gen_len'],
                'Diversity_rep2': data['prediction_dive'],
                'Diversity_rep3': data['prediction_dive'],
                'Diversity_rep4': data['prediction_dive'],
            }
        except Exception as e:
            print(f"Impossible de traiter {f_path}: {e}")

    # Trouve les fichiers de cohérence et les fusionne
    coherence_files = glob.glob(os.path.join(base_dir, '**/*_coherence_result.json'), recursive=True)

    for c_path in coherence_files:
        try:
            dataset, strategy = parse_filename(c_path)
            key = (dataset, strategy)
            if key not in all_results_map:
                all_results_map[key] = {'Dataset': dataset, 'Strategy': strategy}
                
            with open(c_path, 'r') as f:
                c_data = json.load(f)[0]
            all_results_map[key]['Coherence'] = float(c_data['coherence_mean'])
        except Exception as e:
            print(f'Impossible de traiter le fichier de cohérence {c_path}: {e}')

    # Finalisation du DataFrame
    final_list = list(all_results_map.values())
    for res in final_list:
        if 'Coherence' not in res:
            res['Coherence'] = np.nan
            
    return pd.DataFrame(final_list)

In [48]:
results_df = load_all_results()
print("Données chargées avec succès !")
results_df.head()

Données chargées avec succès !


,Dataset,Strategy,MAUVE,Gen_Length,Diversity_rep2,Diversity_rep3,Diversity_rep4,Coherence
0,book,Greedy Search,2.65,194.33,0.86,0.86,0.86,-0.336219
1,book,Nucleus (p=0.95),91.47,203.21,94.50,94.50,94.50,-3.018318
2,book,Typical (p=0.95),88.58,199.01,96.29,96.29,96.29,-3.683371
3,wikinews,Greedy Search,13.96,195.23,3.55,3.55,3.55,-0.467460
4,wikinews,Nucleus (p=0.95),89.45,195.98,93.54,93.54,93.54,-2.606337


## 3. Tableau Comparatif

Voici le tableau complet des résultats, trié par dataset et stratégie. C'est la vue la plus dense pour comparer numériquement les performances.

In [49]:
full_table = results_df.set_index(['Dataset', 'Strategy']).sort_index()
display(full_table)

MAUVE  Gen_Length  Diversity_rep2  Diversity_rep3  Diversity_rep4  Coherence
Dataset  Strategy                                                                                      
book     Greedy Search      2.65      194.33            0.86            0.86            0.86  -0.336219
         Nucleus (p=0.95)  91.47      203.21           94.50           94.50           94.50  -3.018318
         Typical (p=0.95)  88.58      199.01           96.29           96.29           96.29  -3.683371
wikinews Greedy Search     13.96      195.23            3.55            3.55            3.55  -0.467460
         Nucleus (p=0.95)  89.45      195.98           93.54           93.54           93.54  -2.606337
         Typical (p=0.95)  90.97      192.68           95.37           95.37           95.37  -3.257353
wikitext Greedy Search      4.91      193.86            1.77            1.77            1.77  -0.406995
         Nucleus (p=0.95)  86.54      192.23           92.16           92.16           92.16  -3.029777
         Typical (p=0.95)  86.07      188.13           94.82           94.82           94.82  -3.709736

## 4. Analyse Visuelle (Graphiques)

Les graphiques permettent de visualiser rapidement les tendances et les compromis. Nous allons créer une fonction réutilisable pour afficher chaque métrique.

In [50]:
def plot_grouped_bar(df, metric, title, higher_is_better=True, output_filename=None):
    """Crée un graphique à barres groupées pour une métrique donnée."""
    pivot_df = df.pivot(index='Dataset', columns='Strategy', values=metric)
    
    ax = pivot_df.plot(kind='bar', figsize=(12, 7), rot=0, colormap='viridis')
    
    # Ajout des valeurs sur les barres
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', label_type='edge', padding=2)
        
    plt.title(title, fontsize=16)
    direction = "(plus haut = mieux)" if higher_is_better else "(plus bas = mieux)"
    plt.ylabel(f"{metric} {direction}", fontsize=12)
    plt.xlabel('Dataset', fontsize=12)
    plt.legend(title='Stratégie', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    if output_filename:
        plt.savefig(output_filename)
        print(f"Graphique sauvegardé : {output_filename}")
    else:
        plt.show()
    
    plt.close() # Libère la mémoire

### Cohérence

La cohérence mesure à quel point le texte généré est plausible et suit le contexte. Un score plus élevé est meilleur.

In [51]:
plot_grouped_bar(results_df, 'Coherence', 'Comparaison de la Cohérence', higher_is_better=True, output_filename='comparaison_coherence.png')

Graphique sauvegardé : comparaison_coherence.png


### MAUVE

MAUVE compare la distribution du texte généré à celle du texte humain de référence. Un score plus élevé indique une plus grande similarité, ce qui est généralement mieux.

In [52]:
plot_grouped_bar(results_df, 'MAUVE', 'Comparaison du score MAUVE', higher_is_better=True, output_filename='comparaison_mauve.png')

Graphique sauvegardé : comparaison_mauve.png


### Diversité (Répétition des N-grams)

Cette métrique mesure le pourcentage de répétition des n-grams (groupes de 2, 3 ou 4 mots). **Un score plus bas est meilleur**, car il indique moins de répétitions et donc plus de diversité.

In [53]:
plot_grouped_bar(results_df, 'Diversity_rep2', 'Taux de Répétition des Bi-grams (rep-2)', higher_is_better=False, output_filename='comparaison_rep2.png')
plot_grouped_bar(results_df, 'Diversity_rep3', 'Taux de Répétition des Tri-grams (rep-3)', higher_is_better=False, output_filename='comparaison_rep3.png')
plot_grouped_bar(results_df, 'Diversity_rep4', 'Taux de Répétition des Quad-grams (rep-4)', higher_is_better=False, output_filename='comparaison_rep4.png')

Graphique sauvegardé : comparaison_rep2.png
Graphique sauvegardé : comparaison_rep3.png
Graphique sauvegardé : comparaison_rep4.png


In [8]:
# import subprocess, sys
# r = subprocess.run([sys.executable, 'analysis2.py'], capture_output=True, text=True)
# print(r.stdout)
# if r.stderr:print("Erreurs :", r.stderr)
!python analysis2.py

Analyzing results from: open_text_gen/wikitext_grid_search

Grid Search Results Analysis (Colored)
Red = Best in Column | Blue = Best Row per Model | Green = Best Row Overall
Model        k   alpha   Coh_gpt2  Coh_gpt2-large  Coh_gpt2-medium  Coh_gpt2-xl  Coh_opt-1.3b  Coh_opt-125m  Coh_opt-2.7b  MAUVE    Gen_Length  rep-2   rep-3   rep-4   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
gpt2         5   0.4000  -1.0036   -1.5675         -1.0137          -1.2978      -0.8858       -1.0346       -0.8588       5.4600   90.6200     0.8682  0.8268  0.7954  
gpt2         5   0.6000  -1.0707   -1.8471         -1.0742          -1.4810      -0.9282       -1.0877       -0.8988       1.1900   71.0900     0.8159  0.7629  0.7170  
gpt2         5   0.8000  -1.1255   -2.1786         -1.1388          -1.7167      -0.9486       -1.1303       -0.9136       1.2800   45.5100     0.716

In [1]:
!python analysis3.py

Analyzing all results from: open_text_gen/wikitext_grid_search (Grid Search + Baselines)

Consolidated Results: Grid Search vs Baselines
Red = Best in Column | Blue = Best Row per Model | Green = Best Row Overall (based on MAUVE)
Model        Method       Parameters   MAUVE    Gen_Length  rep-2   rep-3   rep-4   Coh_gpt2  Coh_gpt2-large  Coh_gpt2-medium  Coh_gpt2-xl  Coh_opt-1.3b  Coh_opt-125m  Coh_opt-2.7b  
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
gpt2         Contrastive  k=10, a=0.4  3.4200   90.3600     0.8533  0.8090  0.7745  -1.1207   -1.6835         -1.1248          -1.4209      -0.9934       -1.1265       -0.9605       
gpt2         Contrastive  k=10, a=0.6  1.4400   60.0000     0.7459  0.6839  0.6413  -1.1724   -2.0421         -1.1640          -1.6741      -1.0013       -1.1750       -0.9624       
gpt2         Contrastive  k=10, a=0.8 

In [2]:
!python analysis3.py

Analyzing all results from: open_text_gen/wikitext_grid_search (Grid Search + Baselines)

Consolidated Results: Grid Search vs Baselines
Red = Best in Column | Blue = Best Row per Model | Green = Best Row Overall (based on MAUVE)
Model        Method       Parameters   MAUVE    Gen_Length  rep-2   rep-3   rep-4   Coh_gpt2  Coh_gpt2-large  Coh_gpt2-medium  Coh_gpt2-xl  Coh_opt-1.3b  Coh_opt-125m  Coh_opt-2.7b  
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
gpt2         Contrastive  k=10, a=0.4  3.4200   90.3600     0.8533  0.8090  0.7745  -1.1207   -1.6835         -1.1248          -1.4209      -0.9934       -1.1265       -0.9605       
gpt2         Contrastive  k=10, a=0.6  1.4400   60.0000     0.7459  0.6839  0.6413  -1.1724   -2.0421         -1.1640          -1.6741      -1.0013       -1.1750       -0.9624       
gpt2         Contrastive  k=10, a=0.8 

## 5. Conclusion

Ce notebook vous fournit les outils pour comparer objectivement les différentes stratégies de décodage.

**Pistes d'interprétation :**
*   **Compromis Cohérence vs. Diversité :** Vous remarquerez probablement que les stratégies qui excellent en cohérence (comme *Greedy Search*) ont tendance à avoir une faible diversité (taux de répétition élevé). À l'inverse, les méthodes comme *Nucleus Sampling* produisent un texte plus varié, mais peuvent parfois perdre en cohérence.
*   **Le "meilleur" modèle :** Il n'y a souvent pas un seul gagnant. Le meilleur choix dépend de l'application. Pour des tâches créatives, la diversité est clé. Pour des tâches factuelles, la cohérence est reine.

Vous pouvez maintenant utiliser ce notebook comme base pour ajouter de nouveaux résultats d'expériences (par exemple, avec de nouveaux hyperparamètres ou de nouveaux modèles) et les comparer facilement.